# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol
import psycopg2

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')

In [17]:
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')

# Load

## 등록정보(STD_CEG_CAR_MIG)

In [6]:
# exasol db
# 4m 21s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG;")
# test 만개
# car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE ROWNUM <= 10000;")

C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


In [7]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [8]:
carr = car.rename(columns=car_ch_col)

In [9]:
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [10]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")

In [11]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [12]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [13]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47916 entries, 0 to 47915
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       47916 non-null  int64 
 1   STDG_CTPV_NM  47916 non-null  object
 2   STDG_SGG_NM   47892 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [14]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [15]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [16]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## 등록이력(CEG_CAR_HISTORY_MIG)

In [18]:
# 1m 13.2s
his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

In [19]:
# edb_id = 'vsysd'
# edb_database = 'edb'
# edb_port = 5444
# edb_url = '172.29.135.50'
# edb_pwd = 'vsyswynn'
# conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
# cur = conn.cursor()
# sql = 'select VHCL_ERSR_YN, CHNG_DE, VHMNO FROM from vsysd.ceg_car_history_mig'
# cur.execute(sql)
# his = pd.DataFrame(cur.fetchall())
# his.columns = [desc[0].upper() for desc in cur.description]
# cur.close()
# conn.close()

# 전처리

## 중복 차대번호 제거

In [20]:
carr.shape, len(carr['차대번호'].unique())

((29998033, 11), 29984639)

In [21]:
carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

## 배출가스등급 코드 변환

In [ ]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['X', '2', '1', '3', '5', '4', nan], dtype=object)

In [ ]:
carr['배출가스등급'].value_counts(dropna=False)

배출가스등급
2      12174609
3      10050952
1       2940294
5       2377906
4       1897506
X        543341
NaN          31
Name: count, dtype: int64

## 차종 코드 변환

In [ ]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['승용', '화물', '승합', '특수', nan], dtype=object)

## 용도 코드 변환

In [ ]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [ ]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(26203794, 11)

## 연료 코드 변환

In [ ]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', '태양열', 'CNG 하이브리드', '등유', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [ ]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [ ]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(26203794, 17)

## 등록&제원&법정동코드 병합(df)

In [ ]:
df = cs.merge(coder, on='법정동코드', how='left')

In [ ]:
df.shape

(26203794, 19)

### 매칭 안되는 지역 처리

In [ ]:
df['시도'].isnull().sum()

1809

In [ ]:
df.loc[df['시도'].isnull() == True, ['법정동코드', '시도', '시군구']]

,법정동코드,시도,시군구
350,4161011500,NaN,NaN
630,4161012000,NaN,NaN
1886,4161011800,NaN,NaN
3186,4161011800,NaN,NaN
4241,4161011900,NaN,NaN
...,...,...,...
838878,4161011500,NaN,NaN
839043,4161011600,NaN,NaN
840902,4161011500,NaN,NaN
25189723,4165052000,NaN,NaN


In [ ]:
df.loc[df['시도'].isnull() == True, '법정동코드'].unique()

array([4161011500, 4161012000, 4161011800, 4161011900, 4161011600,
       4161011700, 4161011400, 4165052000, 4163055000], dtype=int64)

In [ ]:
# 주소 수정
df.loc[df['법정동코드'] == 4161011600, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 능평동
df.loc[df['법정동코드'] == 4161012000, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 양벌동
df.loc[df['법정동코드'] == 4161011800, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 추자동
df.loc[df['법정동코드'] == 4161011500, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 신현동
df.loc[df['법정동코드'] == 4161011900, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 매산동
df.loc[df['법정동코드'] == 4161011700, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 문형동
df.loc[df['법정동코드'] == 4161011400, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 고산동
df.loc[df['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동

In [ ]:
df['시도'].isnull().sum()

0

## 등록(말소 유지) & 제원 병합

In [ ]:
# 10.3s
cse = carr.merge(srcr, on='제원관리번호', how='left')

In [ ]:
cse.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'],
      dtype='object')

## 등록&제원&이력 병합

In [ ]:
# 2m 6.0s
ersr = cse.merge(hisr, on='차량관리번호', how='left')

In [ ]:
ersr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN_x', '제원관리번호', '차량연식',
       '차종', '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식',
       '엔진형식', '차량말소YN_y', '변경일자'],
      dtype='object')

In [ ]:
# 1. 등록 차량말소와 등록이력 차량 말소 둘 모두 해당되는 데이터 추출
ersr = ersr.loc[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)

In [ ]:
# 2. 변경일자 기준 최신 데이터만 남기고 차대번호 중복 제거
ersr = ersr.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [ ]:
# 3. 변경일자 2019.01.01 이후만 추출
ersr = ersr[ersr['변경일자'] >= 20190101].reset_index(drop=True)

## 등록&제원&이력&법정동코드 병합(errc)

In [ ]:
# 매칭 후 '시도' 빈값 없음
errc = ersr.merge(coder, on='법정동코드', how='left')

# 분석

## 연료 하이브리드 수정

In [ ]:
# about 23.6s
df.loc[(df['연료'] == '휘발유') | (df['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
df.loc[(df['연료'] == '경유') | (df['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
df.loc[(df['연료'] == 'LPG(액화석유가스)') | (df['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
df.loc[(df['연료'] == 'CNG(압축천연가스)') | (df['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
df.loc[(df['연료'] == 'LNG(액화천연가스)') | (df['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
df.loc[df['연료'] == '전기', 'fuel'] = '전기'
df.loc[df['연료'] == '수소', 'fuel'] = '수소'
df.loc[df['연료'] == '태양열', 'fuel'] = '태양열'
df.loc[df['연료'] == '알코올', 'fuel'] = '알코올'
df.loc[df['연료'] == '등유', 'fuel'] = '등유'
df.loc[df['연료'] == '기타연료', 'fuel'] = '기타연료'
df.loc[df['연료'].isnull() == True, 'fuel'] = np.nan

In [ ]:
# about 3.3s
errc.loc[(errc['연료'] == '휘발유') | (errc['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
errc.loc[(errc['연료'] == '경유') | (errc['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
errc.loc[(errc['연료'] == 'LPG(액화석유가스)') | (errc['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
errc.loc[(errc['연료'] == 'CNG(압축천연가스)') | (errc['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
errc.loc[(errc['연료'] == 'LNG(액화천연가스)') | (errc['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
errc.loc[errc['연료'] == '전기', 'fuel'] = '전기'
errc.loc[errc['연료'] == '수소', 'fuel'] = '수소'
errc.loc[errc['연료'] == '태양열', 'fuel'] = '태양열'
errc.loc[errc['연료'] == '알코올', 'fuel'] = '알코올'
errc.loc[errc['연료'] == '등유', 'fuel'] = '등유'
errc.loc[errc['연료'] == '기타연료', 'fuel'] = '기타연료'
errc.loc[errc['연료'].isnull() == True, 'fuel'] = np.nan

In [ ]:
# 6s
# 수도권 : 서울특별시, 인천광역시, 경기도
# 비수도권 : 수도권 외
df.loc[(df['시도'] == '서울특별시') | (df['시도'] == '인천광역시') | (df['시도'] == '경기도'), '지역'] = '수도권'
df['지역'] = df['지역'].fillna('비수도권')

In [ ]:
dfm = df.copy()

In [ ]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

In [ ]:
# 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

In [ ]:
# 수도권 : 서울특별시, 인천광역시, 경기도
# 비수도권 : 수도권 외
errc.loc[(errc['시도'] == '서울특별시') | (errc['시도'] == '인천광역시') | (errc['시도'] == '경기도'), '지역'] = '수도권'
errc['지역'] = errc['지역'].fillna('비수도권')

In [ ]:
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

In [ ]:
# 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

In [ ]:
# 2m 41.5s
dfm2 = dfm.loc[(dfm['fuel'] == '경유') | (dfm['fuel'] == '휘발유') | (dfm['fuel'] == 'LPG') | (dfm['fuel'] == '전기') | (dfm['fuel'] == '수소')].reset_index(drop=True)
errc2 = errc.loc[(errc['fuel'] == '경유') | (errc['fuel'] == '휘발유') | (errc['fuel'] == 'LPG') | (errc['fuel'] == '전기') | (errc['fuel'] == '수소')].reset_index(drop=True)

## 등급, 지역별 차량현황

In [ ]:
# 2022년 차량 대수
grp1 = dfm2.groupby(['지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', '지역', '시도', '배출가스등급', '차량대수']]
grp1

,연도,지역,시도,배출가스등급,차량대수
0,2022,비수도권,강원도,1,80078
1,2022,비수도권,강원도,2,358656
2,2022,비수도권,강원도,3,310061
3,2022,비수도권,강원도,4,54678
4,2022,비수도권,강원도,5,47172
...,...,...,...,...,...
97,2022,수도권,인천광역시,2,793986
98,2022,수도권,인천광역시,3,580695
99,2022,수도권,인천광역시,4,77386
100,2022,수도권,인천광역시,5,52890


In [ ]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
rgn_list = []
ctpv_list = []
grd_list = []
for ctpv in grp1['시도'].unique():
    if ('서울특별시' in ctpv) or ('인천광역시' in ctpv) or ('경기도' in ctpv):
        rgn = '수도권'
    else:
        rgn = '비수도권'
        for grd in ['1', '2', '3', '4', '5', 'X']:
            for yr in ['2019', '2020', '2021', '2022']:
                yr_list.append(yr)
                rgn_list.append(rgn)
                ctpv_list.append(ctpv)
                grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, '지역':rgn_list, '시도':ctpv_list, '배출가스등급':grd_list})

In [ ]:
base.head()

,연도,지역,시도,배출가스등급
0,2019,비수도권,강원도,1
1,2020,비수도권,강원도,1
2,2021,비수도권,강원도,1
3,2022,비수도권,강원도,1
4,2019,비수도권,강원도,2


In [ ]:
# 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', '지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,지역,시도,배출가스등급,등록대수
0,1955,비수도권,강원도,5,1
1,1958,비수도권,경상북도,5,1
2,1962,비수도권,강원도,5,4
3,1962,비수도권,경상북도,5,1
4,1962,비수도권,전라북도,5,1
...,...,...,...,...,...
3068,2023,수도권,서울특별시,X,29242
3069,2023,수도권,인천광역시,1,8305
3070,2023,수도권,인천광역시,2,14541
3071,2023,수도권,인천광역시,3,4722


In [ ]:
# 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', '지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,지역,시도,배출가스등급,말소대수
0,2019,비수도권,강원도,1,220
1,2019,비수도권,강원도,2,4208
2,2019,비수도권,강원도,3,8191
3,2019,비수도권,강원도,4,2558
4,2019,비수도권,강원도,5,11783
...,...,...,...,...,...
401,2022,수도권,인천광역시,2,9878
402,2022,수도권,인천광역시,3,10842
403,2022,수도권,인천광역시,4,3113
404,2022,수도권,인천광역시,5,4507


In [ ]:
base1 = base.merge(grp1, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base1

,연도,지역,시도,배출가스등급,차량대수
0,2019,비수도권,강원도,1,NaN
1,2020,비수도권,강원도,1,NaN
2,2021,비수도권,강원도,1,NaN
3,2022,비수도권,강원도,1,80078.00
4,2019,비수도권,강원도,2,NaN
...,...,...,...,...,...
331,2022,비수도권,충청북도,5,40793.00
332,2019,비수도권,충청북도,X,NaN
333,2020,비수도권,충청북도,X,NaN
334,2021,비수도권,충청북도,X,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base2

,연도,지역,시도,배출가스등급,차량대수,등록대수
0,2019,비수도권,강원도,1,NaN,7882.00
1,2020,비수도권,강원도,1,NaN,11940.00
2,2021,비수도권,강원도,1,NaN,14037.00
3,2022,비수도권,강원도,1,80078.00,10891.00
4,2019,비수도권,강원도,2,NaN,23234.00
...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,40793.00,1.00
332,2019,비수도권,충청북도,X,NaN,27.00
333,2020,비수도권,충청북도,X,NaN,38.00
334,2021,비수도권,충청북도,X,NaN,29.00


In [ ]:
base3 = base2.merge(grp3, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base3

,연도,지역,시도,배출가스등급,차량대수,등록대수,말소대수
0,2019,비수도권,강원도,1,NaN,7882.00,220.00
1,2020,비수도권,강원도,1,NaN,11940.00,461.00
2,2021,비수도권,강원도,1,NaN,14037.00,854.00
3,2022,비수도권,강원도,1,80078.00,10891.00,501.00
4,2019,비수도권,강원도,2,NaN,23234.00,4208.00
...,...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,40793.00,1.00,4901.00
332,2019,비수도권,충청북도,X,NaN,27.00,37.00
333,2020,비수도권,충청북도,X,NaN,38.00,26.00
334,2021,비수도권,충청북도,X,NaN,29.00,13.00


In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,지역,시도,배출가스등급,차량대수,등록대수,말소대수
0,2019,비수도권,강원도,1,45026.00,7882.00,220.00
1,2020,비수도권,강원도,1,56505.00,11940.00,461.00
2,2021,비수도권,강원도,1,69688.00,14037.00,854.00
3,2022,비수도권,강원도,1,80078.00,10891.00,501.00
4,2019,비수도권,강원도,2,315866.00,23234.00,4208.00
...,...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,40793.00,1.00,4901.00
332,2019,비수도권,충청북도,X,8344.00,27.00,37.00
333,2020,비수도권,충청북도,X,8356.00,38.00,26.00
334,2021,비수도권,충청북도,X,8372.00,29.00,13.00


In [ ]:
base3.columns

Index(['연도', '지역', '시도', '배출가스등급', '차량대수', '등록대수', '말소대수'], dtype='object')

In [ ]:
df1 = base3[['연도', '지역', '시도', '배출가스등급', '차량대수']]

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

In [ ]:
df1['테이블생성일자'] = today_date

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '지역':'RGN', 
    '시도':'CTPV', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}

In [ ]:
STD_BD_CAR_CURSTT_MOD = df1.rename(columns=cdict)
STD_BD_CAR_CURSTT_MOD.columns

Index(['YR', 'RGN', 'CTPV', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT'], dtype='object')

In [ ]:
STD_BD_CAR_CURSTT_MOD.head()

,YR,RGN,CTPV,EXHST_GAS_GRD_CD,VHCL_MKCNT
0,2019,비수도권,강원도,1,45026.00
1,2020,비수도권,강원도,1,56505.00
2,2021,비수도권,강원도,1,69688.00
3,2022,비수도권,강원도,1,80078.00
4,2019,비수도권,강원도,2,315866.00


### [출력] STD_BD_CAR_CURSTT_MOD

In [ ]:
# expdf = STD_BD_CAR_CURSTT_MOD
# table_nm = 'STD_BD_CAR_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

## 등급, 연료별 차량현황

In [ ]:
# 2022년 차량 대수
grp1 = dfm2.groupby(['fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', 'fuel', '배출가스등급', '차량대수']]

In [ ]:
grp1.head()

,연도,fuel,배출가스등급,차량대수
0,2022,LPG,1,479004
1,2022,LPG,2,1176911
2,2022,LPG,3,238483
3,2022,LPG,4,24549
4,2022,LPG,5,436


In [ ]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
grd_list = []
for fuel in grp1['fuel'].unique():
    for grd in ['1', '2', '3', '4', '5', 'X']:
        for yr in ['2019', '2020', '2021', '2022']:
            yr_list.append(yr)
            fuel_list.append(fuel)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list})

In [ ]:
base.head()

,연도,fuel,배출가스등급
0,2019,LPG,1
1,2020,LPG,1
2,2021,LPG,1
3,2022,LPG,1
4,2019,LPG,2


In [ ]:
# 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,배출가스등급,등록대수
0,1955,경유,5,1
1,1958,휘발유,5,1
2,1962,경유,5,8
3,1962,휘발유,5,1
4,1963,경유,5,2
...,...,...,...,...
476,2023,전기,1,34858
477,2023,전기,X,15135
478,2023,휘발유,1,72399
479,2023,휘발유,2,157626


In [ ]:
# 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,배출가스등급,말소대수
0,2019,LPG,1,5974
1,2019,LPG,2,27863
2,2019,LPG,3,42824
3,2019,LPG,4,3458
4,2019,LPG,5,3
...,...,...,...,...
71,2022,휘발유,2,116722
72,2022,휘발유,3,94401
73,2022,휘발유,4,11563
74,2022,휘발유,5,206


In [ ]:
base1 = base.merge(grp1, on=['연도', 'fuel', '배출가스등급'], how='left')
base1

,연도,fuel,배출가스등급,차량대수
0,2019,LPG,1,NaN
1,2020,LPG,1,NaN
2,2021,LPG,1,NaN
3,2022,LPG,1,479004.00
4,2019,LPG,2,NaN
...,...,...,...,...
115,2022,휘발유,5,30012.00
116,2019,휘발유,X,NaN
117,2020,휘발유,X,NaN
118,2021,휘발유,X,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', 'fuel', '배출가스등급'], how='left')
base2

,연도,fuel,배출가스등급,차량대수,등록대수
0,2019,LPG,1,NaN,62978.00
1,2020,LPG,1,NaN,47598.00
2,2021,LPG,1,NaN,36005.00
3,2022,LPG,1,479004.00,16628.00
4,2019,LPG,2,NaN,65541.00
...,...,...,...,...,...
115,2022,휘발유,5,30012.00,NaN
116,2019,휘발유,X,NaN,2660.00
117,2020,휘발유,X,NaN,4611.00
118,2021,휘발유,X,NaN,5592.00


In [ ]:
base3 = base2.merge(grp3, on=['연도', 'fuel', '배출가스등급'], how='left')
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,NaN,62978.00,5974.00
1,2020,LPG,1,NaN,47598.00,15887.00
2,2021,LPG,1,NaN,36005.00,23771.00
3,2022,LPG,1,479004.00,16628.00,10239.00
4,2019,LPG,2,NaN,65541.00,27863.00
...,...,...,...,...,...,...
115,2022,휘발유,5,30012.00,NaN,206.00
116,2019,휘발유,X,NaN,2660.00,303.00
117,2020,휘발유,X,NaN,4611.00,774.00
118,2021,휘발유,X,NaN,5592.00,1931.00


In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,428670.00,62978.00,5974.00
1,2020,LPG,1,460381.00,47598.00,15887.00
2,2021,LPG,1,472615.00,36005.00,23771.00
3,2022,LPG,1,479004.00,16628.00,10239.00
4,2019,LPG,2,1102289.00,65541.00,27863.00
...,...,...,...,...,...,...
115,2022,휘발유,5,30012.00,0.00,206.00
116,2019,휘발유,X,174659.00,2660.00,303.00
117,2020,휘발유,X,178496.00,4611.00,774.00
118,2021,휘발유,X,182157.00,5592.00,1931.00


In [ ]:
df2 = base3[['연도', 'fuel', '배출가스등급', '차량대수']]

In [ ]:
df2['테이블생성일자'] = today_date

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}

In [ ]:
STD_BD_CAR_CURSTT_MOD2 = df2.rename(columns=cdict)
STD_BD_CAR_CURSTT_MOD2.columns

Index(['YR', 'fuel', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT'], dtype='object')

In [ ]:
STD_BD_CAR_CURSTT_MOD2.head()

,YR,fuel,EXHST_GAS_GRD_CD,VHCL_MKCNT
0,2019,LPG,1,428670.00
1,2020,LPG,1,460381.00
2,2021,LPG,1,472615.00
3,2022,LPG,1,479004.00
4,2019,LPG,2,1102289.00


### [출력] STD_BD_CAR_CURSTT_MOD2

In [ ]:
# expdf = STD_BD_CAR_CURSTT_MOD2
# table_nm = 'STD_BD_CAR_CURSTT_MOD2'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

## 전체 차량 현황(등급, 차종, 지역별 차량현황)

In [ ]:
dfm2['테이블생성일자'] = today_date

In [ ]:
df3 = dfm2[[
    '테이블생성일자', 
    '법정동코드', 
    '차종',
    '용도', 
    '차대번호', 
    '제원관리번호', 
    '배출가스인증번호', 
    '배출가스등급', 
    '차량말소YN', 
    'fuel', 
    '시도', 
    '시군구_수정'
]]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT',
    '법정동코드':'STDG_CD', 
    '차종':'VHCTY_CD',
    '용도':'PURPS_CD2', 
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량말소YN':'VHCL_ERSR_YN', 
    'fuel':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
}

In [ ]:
STD_BD_CAR_CURSTT = df3.rename(columns=cdict)
STD_BD_CAR_CURSTT.columns

Index(['STDG_CD', 'VHCTY_CD', 'PURPS_CD2', 'VIN', 'MANG_MNG_NO',
       'EXHST_GAS_CERT_NO', 'EXHST_GAS_GRD_CD', 'VHCL_ERSR_YN', 'FUEL_CD',
       'CTPV', 'SGG'],
      dtype='object')

### [출력] STD_BD_CAR_CURSTT

In [ ]:
# expdf = STD_BD_CAR_CURSTT
# table_nm = 'STD_BD_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 3m 51.1s
# we.import_from_pandas(expdf, table_nm)

## 지역, 연료, 연도별 차량 현황 분석

In [ ]:
# 2022년 차량 대수
grp1 = dfm2.groupby(['fuel', '시도'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', 'fuel', '시도', '차량대수']]

In [ ]:
grp1.head()

,연도,fuel,시도,차량대수
0,2022,LPG,강원도,59042
1,2022,LPG,경기도,401770
2,2022,LPG,경상남도,142124
3,2022,LPG,경상북도,129463
4,2022,LPG,광주광역시,90115


In [ ]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
ctpv_list = []
for fuel in grp1['fuel'].unique():
    for ctpv in grp1['시도'].unique():
        for yr in ['2019', '2020', '2021', '2022']:
            yr_list.append(yr)
            fuel_list.append(fuel)
            ctpv_list.append(ctpv)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '시도':ctpv_list})

In [ ]:
base.head()

,연도,fuel,시도
0,2019,LPG,강원도
1,2020,LPG,강원도
2,2021,LPG,강원도
3,2022,LPG,강원도
4,2019,LPG,경기도


In [ ]:
# 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', 'fuel', '시도'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,시도,등록대수
0,1955,경유,강원도,1
1,1958,휘발유,경상북도,1
2,1962,경유,강원도,4
3,1962,경유,경기도,2
4,1962,경유,전라북도,1
...,...,...,...,...
2765,2023,휘발유,전라남도,22196
2766,2023,휘발유,전라북도,11390
2767,2023,휘발유,제주특별자치도,19860
2768,2023,휘발유,충청남도,16292


In [ ]:
# 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', 'fuel', '시도'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,시도,말소대수
0,2019,LPG,강원도,2719
1,2019,LPG,경기도,17972
2,2019,LPG,경상남도,5673
3,2019,LPG,경상북도,4960
4,2019,LPG,광주광역시,3067
...,...,...,...,...
303,2022,휘발유,전라남도,8254
304,2022,휘발유,전라북도,8175
305,2022,휘발유,제주특별자치도,2989
306,2022,휘발유,충청남도,11178


In [ ]:
base1 = base.merge(grp1, on=['연도', 'fuel', '시도'], how='left')
base1

,연도,fuel,시도,차량대수
0,2019,LPG,강원도,NaN
1,2020,LPG,강원도,NaN
2,2021,LPG,강원도,NaN
3,2022,LPG,강원도,59042.00
4,2019,LPG,경기도,NaN
...,...,...,...,...
335,2022,휘발유,충청남도,587731.00
336,2019,휘발유,충청북도,NaN
337,2020,휘발유,충청북도,NaN
338,2021,휘발유,충청북도,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', 'fuel', '시도'], how='left')
base2

,연도,fuel,시도,차량대수,등록대수
0,2019,LPG,강원도,NaN,3386.00
1,2020,LPG,강원도,NaN,3433.00
2,2021,LPG,강원도,NaN,3003.00
3,2022,LPG,강원도,59042.00,1951.00
4,2019,LPG,경기도,NaN,24300.00
...,...,...,...,...,...
335,2022,휘발유,충청남도,587731.00,31172.00
336,2019,휘발유,충청북도,NaN,29768.00
337,2020,휘발유,충청북도,NaN,35756.00
338,2021,휘발유,충청북도,NaN,31143.00


In [ ]:
base3 = base2.merge(grp3, on=['연도', 'fuel', '시도'], how='left')
base3

,연도,fuel,시도,차량대수,등록대수,말소대수
0,2019,LPG,강원도,NaN,3386.00,2719.00
1,2020,LPG,강원도,NaN,3433.00,4651.00
2,2021,LPG,강원도,NaN,3003.00,4552.00
3,2022,LPG,강원도,59042.00,1951.00,1883.00
4,2019,LPG,경기도,NaN,24300.00,17972.00
...,...,...,...,...,...,...
335,2022,휘발유,충청남도,587731.00,31172.00,11178.00
336,2019,휘발유,충청북도,NaN,29768.00,9895.00
337,2020,휘발유,충청북도,NaN,35756.00,18006.00
338,2021,휘발유,충청북도,NaN,31143.00,18139.00


In [ ]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    255
등록대수      3
말소대수     32
dtype: int64

In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,fuel,시도,차량대수,등록대수,말소대수
0,2019,LPG,강원도,61741.00,3386.00,2719.00
1,2020,LPG,강원도,60523.00,3433.00,4651.00
2,2021,LPG,강원도,58974.00,3003.00,4552.00
3,2022,LPG,강원도,59042.00,1951.00,1883.00
4,2019,LPG,경기도,423688.00,24300.00,17972.00
...,...,...,...,...,...,...
335,2022,휘발유,충청남도,587731.00,31172.00,11178.00
336,2019,휘발유,충청북도,404608.00,29768.00,9895.00
337,2020,휘발유,충청북도,422358.00,35756.00,18006.00
338,2021,휘발유,충청북도,435362.00,31143.00,18139.00


In [ ]:
gp = base3.groupby(['연도', '시도', 'fuel'], as_index=False)['차량대수'].sum()

In [ ]:
gp['연료비율'] = round((gp['차량대수'] / gp.groupby(['연도', '시도'])['차량대수'].transform('sum')), 2)

In [ ]:
gp['테이블생성일자'] = today_date

In [ ]:
gp1 = gp[['테이블생성일자', '연도', 'fuel', '시도', '차량대수', '연료비율']]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    'fuel':'FUEL_CD', 
    '시도':'CTPV', 
    '차량대수':'VHCL_MKCNT', 
    '연료비율':'FUEL_RT'
}

In [ ]:
STD_BD_CAR_REG_MKCNT = gp1.rename(columns=cdict)
STD_BD_CAR_REG_MKCNT.columns

Index(['YR', 'FUEL_CD', 'CTPV', 'VHCL_MKCNT', 'FUEL_RT'], dtype='object')

In [ ]:
STD_BD_CAR_REG_MKCNT.head()

,YR,FUEL_CD,CTPV,VHCL_MKCNT,FUEL_RT
0,2019,LPG,강원도,61741.00,0.08
1,2019,경유,강원도,386625.00,0.47
2,2019,수소,강원도,262.00,0.00
3,2019,전기,강원도,3833.00,0.00
4,2019,휘발유,강원도,369913.00,0.45


### [출력] STD_BD_CAR_REG_MKCNT

In [ ]:
# expdf = STD_BD_CAR_REG_MKCNT
# table_nm = 'STD_BD_CAR_REG_MKCNT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

## 내연차 연료, 연도별 차량 현황 예측
- 경유, 휘발유, LPG

In [ ]:
dfm2dgl = dfm2.loc[(dfm2['연료'] == '경유') | (dfm2['연료'] == '휘발유') | (dfm2['연료'] == 'LPG')].reset_index(drop=True)
dfm2dgl.shape

(25468466, 25)

In [ ]:
errc2dgl = errc2.loc[(errc2['연료'] == '경유') | (errc2['연료'] == '휘발유') | (errc2['연료'] == 'LPG')].reset_index(drop=True)
errc2dgl.shape

(3598711, 27)

In [ ]:
# 2022년 차량 대수
grp1 = dfm2dgl.groupby('연료', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', '연료', '차량대수']]

In [ ]:
grp1.head()

,연도,fuel,차량대수
0,2022,LPG,1936071
1,2022,경유,9980221
2,2022,휘발유,13552174


In [ ]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['연료'].unique():
    for yr in ['2019', '2020', '2021', '2022']:
        yr_list.append(yr)
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list})

In [ ]:
base.head()

,연도,fuel
0,2019,LPG
1,2020,LPG
2,2021,LPG
3,2022,LPG
4,2019,경유


In [ ]:
# 연도별 등록대수
grp2 = dfm2dgl.groupby(['최초등록일자_년', '연료'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,등록대수
0,1955,경유,1
1,1958,휘발유,1
2,1962,경유,8
3,1962,휘발유,1
4,1963,경유,2
...,...,...,...
163,2022,경유,283522
164,2022,휘발유,867185
165,2023,LPG,20773
166,2023,경유,109007


In [ ]:
# 연도별 말소대수
grp3 = errc2dgl.groupby(['변경일자_년', '연료'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,말소대수
0,2019,LPG,80281
1,2019,경유,338063
2,2019,휘발유,282565
3,2020,LPG,143468
4,2020,경유,557396
5,2020,휘발유,511870
6,2021,LPG,134849
7,2021,경유,539525
8,2021,휘발유,508910
9,2022,LPG,55989


In [ ]:
base1 = base.merge(grp1, on=['연도', '연료'], how='left')
base1

,연도,fuel,차량대수
0,2019,LPG,NaN
1,2020,LPG,NaN
2,2021,LPG,NaN
3,2022,LPG,1936071.00
4,2019,경유,NaN
5,2020,경유,NaN
6,2021,경유,NaN
7,2022,경유,9980221.00
8,2019,휘발유,NaN
9,2020,휘발유,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', '연료'], how='left')
base2

,연도,fuel,차량대수,등록대수
0,2019,LPG,NaN,128532
1,2020,LPG,NaN,124962
2,2021,LPG,NaN,104597
3,2022,LPG,1936071.00,68245
4,2019,경유,NaN,645387
5,2020,경유,NaN,590066
6,2021,경유,NaN,429316
7,2022,경유,9980221.00,283522
8,2019,휘발유,NaN,946499
9,2020,휘발유,NaN,1116605


In [ ]:
base3 = base2.merge(grp3, on=['연도', '연료'], how='left')
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,LPG,NaN,128532,80281
1,2020,LPG,NaN,124962,143468
2,2021,LPG,NaN,104597,134849
3,2022,LPG,1936071.00,68245,55989
4,2019,경유,NaN,645387,338063
5,2020,경유,NaN,590066,557396
6,2021,경유,NaN,429316,539525
7,2022,경유,9980221.00,283522,216405
8,2019,휘발유,NaN,946499,282565
9,2020,휘발유,NaN,1116605,511870


In [ ]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    9
등록대수    0
말소대수    0
dtype: int64

In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,LPG,1972573.00,128532,80281
1,2020,LPG,1954067.00,124962,143468
2,2021,LPG,1923815.00,104597,134849
3,2022,LPG,1936071.00,68245,55989
4,2019,경유,9990643.00,645387,338063
5,2020,경유,10023313.00,590066,557396
6,2021,경유,9913104.00,429316,539525
7,2022,경유,9980221.00,283522,216405
8,2019,휘발유,11741283.00,946499,282565
9,2020,휘발유,12346018.00,1116605,511870


In [ ]:
die = base3.loc[base3['연료'] == '경유', ['연도', '연료', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['연료'] == '휘발유', ['연도', '연료', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['연료'] == 'LPG', ['연도', '연료', '차량대수']].reset_index(drop=True)

In [ ]:
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [ ]:
fit1 = np.polyfit(die['연도'], die['차량대수'], 1)
fit2 = np.polyfit(gas['연도'], gas['차량대수'], 1)
fit3 = np.polyfit(lpg['연도'], lpg['차량대수'], 1)

In [ ]:
a1, b1 = fit1
a2, b2 = fit2
a3, b3 = fit3

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '경유'
for yr in range(2023, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
die_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '경유_예측':pred_list})

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '휘발유'
for yr in range(2023, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
gas_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '휘발유_예측':pred_list})

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = 'LPG'
for yr in range(2023, 2036):
    pred = a3 * yr + b3
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
lpg_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, 'LPG_예측':pred_list})

In [ ]:
die_t = pd.concat([die, die_pred], ignore_index=True)
gas_t = pd.concat([gas, gas_pred], ignore_index=True)
lpg_t = pd.concat([lpg, lpg_pred], ignore_index=True)

In [ ]:
die_t = die_t.rename(columns={'차량대수':'경유_대수'})
gas_t = gas_t.rename(columns={'차량대수':'휘발유_대수'})
lpg_t = lpg_t.rename(columns={'차량대수':'LPG_대수'})

In [ ]:
die_t.columns

Index(['연도', 'fuel', '경유_대수', '경유_예측'], dtype='object')

In [ ]:
df5 = pd.concat([die_t[['연도', '경유_대수', '경유_예측']], gas_t[['휘발유_대수', '휘발유_예측']], lpg_t[['LPG_대수', 'LPG_예측']]], axis=1)
df5.head()

,연도,경유_대수,경유_예측,휘발유_대수,휘발유_예측,LPG_대수,LPG_예측
0,2019,9990643.00,NaN,11741283.00,NaN,1972573.00,NaN
1,2020,10023313.00,NaN,12346018.00,NaN,1954067.00,NaN
2,2021,9913104.00,NaN,12914379.00,NaN,1923815.00,NaN
3,2022,9980221.00,NaN,13552174.00,NaN,1936071.00,NaN
4,2023,NaN,9941451.50,NaN,14138722.00,NaN,1911692.00


In [ ]:
df5.columns

Index(['연도', '경유_대수', '경유_예측', '휘발유_대수', '휘발유_예측', 'LPG_대수', 'LPG_예측'], dtype='object')

In [ ]:
df5['테이블생성일자'] = today_date

In [ ]:
df5 = df5[[
    '테이블생성일자', 
    '연도', 
    '휘발유_대수', 
    '경유_대수', 
    'LPG_대수', 
    '휘발유_예측', 
    '경유_예측', 
    'LPG_예측'
    ]]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '휘발유_대수':'GSL', 
    '경유_대수':'DSL', 
    'LPG_대수':'LPG', 
    '휘발유_예측':'GSL_PRET', 
    '경유_예측':'DSL_PRET', 
    'LPG_예측':'LPG_PRET',
}

In [ ]:
STD_BD_CAR_PRET = df5.rename(columns=cdict)
STD_BD_CAR_PRET.columns

Index(['YR', 'GSL', 'DSL', 'LPG', 'GSL_PRET', 'DSL_PRET', 'LPG_PRET'], dtype='object')

In [ ]:
STD_BD_CAR_PRET.tail()

,YR,GSL,DSL,LPG,GSL_PRET,DSL_PRET,LPG_PRET
12,2031,NaN,NaN,NaN,18939549.20,9828271.50,1799885.60
13,2032,NaN,NaN,NaN,19539652.60,9814124.00,1785909.80
14,2033,NaN,NaN,NaN,20139756.00,9799976.50,1771934.00
15,2034,NaN,NaN,NaN,20739859.40,9785829.00,1757958.20
16,2035,NaN,NaN,NaN,21339962.80,9771681.50,1743982.40


### [출력] STD_BD_CAR_PRET

In [ ]:
# expdf = STD_BD_CAR_PRET
# table_nm = 'STD_BD_CAR_PRET'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

## 하이브리드 연료, 연도별 차량 현황 예측
- 경유, 휘발유, LPG 하이브리드

In [ ]:
dfm2h = dfm2.loc[(dfm2['연료'] == '경유 하이브리드') | (dfm2['연료'] == '휘발유 하이브리드') | (dfm2['연료'] == 'LPG 하이브리드')].reset_index(drop=True)
dfm2h.shape

(25468466, 25)

In [ ]:
errc2h = errc2.loc[(errc2['연료'] == '경유 하이브리드') | (errc2['연료'] == '휘발유 하이브리드') | (errc2['연료'] == 'LPG 하이브리드')].reset_index(drop=True)
errc2h.shape

(3598711, 27)

In [ ]:
# 2022년 차량 대수
grp1 = dfm2h.groupby('연료', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', '연료', '차량대수']]

In [ ]:
grp1.head()

,연도,fuel,차량대수
0,2022,LPG,1936071
1,2022,경유,9980221
2,2022,휘발유,13552174


In [ ]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['연료'].unique():
    for yr in ['2019', '2020', '2021', '2022']:
        yr_list.append(yr)
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list})

In [ ]:
base.head()

,연도,fuel
0,2019,LPG
1,2020,LPG
2,2021,LPG
3,2022,LPG
4,2019,경유


In [ ]:
# 연도별 등록대수
grp2 = dfm2h.groupby(['최초등록일자_년', '연료'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,등록대수
0,1955,경유,1
1,1958,휘발유,1
2,1962,경유,8
3,1962,휘발유,1
4,1963,경유,2
...,...,...,...
163,2022,경유,283522
164,2022,휘발유,867185
165,2023,LPG,20773
166,2023,경유,109007


In [ ]:
# 연도별 말소대수
grp3 = errc2h.groupby(['변경일자_년', '연료'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,말소대수
0,2019,LPG,80281
1,2019,경유,338063
2,2019,휘발유,282565
3,2020,LPG,143468
4,2020,경유,557396
5,2020,휘발유,511870
6,2021,LPG,134849
7,2021,경유,539525
8,2021,휘발유,508910
9,2022,LPG,55989


In [ ]:
base1 = base.merge(grp1, on=['연도', '연료'], how='left')
base1

,연도,fuel,차량대수
0,2019,LPG,NaN
1,2020,LPG,NaN
2,2021,LPG,NaN
3,2022,LPG,1936071.00
4,2019,경유,NaN
5,2020,경유,NaN
6,2021,경유,NaN
7,2022,경유,9980221.00
8,2019,휘발유,NaN
9,2020,휘발유,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', '연료'], how='left')
base2

,연도,fuel,차량대수,등록대수
0,2019,LPG,NaN,128532
1,2020,LPG,NaN,124962
2,2021,LPG,NaN,104597
3,2022,LPG,1936071.00,68245
4,2019,경유,NaN,645387
5,2020,경유,NaN,590066
6,2021,경유,NaN,429316
7,2022,경유,9980221.00,283522
8,2019,휘발유,NaN,946499
9,2020,휘발유,NaN,1116605


In [ ]:
base3 = base2.merge(grp3, on=['연도', '연료'], how='left')
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,LPG,NaN,128532,80281
1,2020,LPG,NaN,124962,143468
2,2021,LPG,NaN,104597,134849
3,2022,LPG,1936071.00,68245,55989
4,2019,경유,NaN,645387,338063
5,2020,경유,NaN,590066,557396
6,2021,경유,NaN,429316,539525
7,2022,경유,9980221.00,283522,216405
8,2019,휘발유,NaN,946499,282565
9,2020,휘발유,NaN,1116605,511870


In [ ]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    9
등록대수    0
말소대수    0
dtype: int64

In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,LPG,1972573.00,128532,80281
1,2020,LPG,1954067.00,124962,143468
2,2021,LPG,1923815.00,104597,134849
3,2022,LPG,1936071.00,68245,55989
4,2019,경유,9990643.00,645387,338063
5,2020,경유,10023313.00,590066,557396
6,2021,경유,9913104.00,429316,539525
7,2022,경유,9980221.00,283522,216405
8,2019,휘발유,11741283.00,946499,282565
9,2020,휘발유,12346018.00,1116605,511870


In [ ]:
die = base3.loc[base3['연료'] == '경유 하이브리드', ['연도', 'fuel', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['연료'] == '휘발유 하이브리드', ['연도', 'fuel', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['연료'] == 'LPG 하이브리드', ['연도', 'fuel', '차량대수']].reset_index(drop=True)

In [ ]:
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [ ]:
fit1 = np.polyfit(die['연도'], die['차량대수'], 1)
fit2 = np.polyfit(gas['연도'], gas['차량대수'], 1)
fit3 = np.polyfit(lpg['연도'], lpg['차량대수'], 1)

In [ ]:
a1, b1 = fit1
a2, b2 = fit2
a3, b3 = fit3

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '경유 하이브리드'
for yr in range(2023, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
die_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '경유_하이브리드_예측':pred_list})

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '휘발유 하이브리드'
for yr in range(2023, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
gas_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '휘발유_하이브리드_예측':pred_list})

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = 'LPG 하이브리드'
for yr in range(2023, 2036):
    pred = a3 * yr + b3
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
lpg_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, 'LPG_하이브리드_예측':pred_list})

In [ ]:
die_t = pd.concat([die, die_pred], ignore_index=True)
gas_t = pd.concat([gas, gas_pred], ignore_index=True)
lpg_t = pd.concat([lpg, lpg_pred], ignore_index=True)

In [ ]:
die_t = die_t.rename(columns={'차량대수':'경유_하이브리드_대수'})
gas_t = gas_t.rename(columns={'차량대수':'휘발유_하이브리드_대수'})
lpg_t = lpg_t.rename(columns={'차량대수':'LPG_하이브리드_대수'})

In [ ]:
die_t.columns

Index(['연도', 'fuel', '경유_대수', '경유_예측'], dtype='object')

In [ ]:
df5 = pd.concat([die_t[['연도', '경유_하이브리드_대수', '경유_하이브리드_예측']], gas_t[['휘발유_하이브리드_대수', '휘발유_하이브리드_예측']], lpg_t[['LPG_하이브리드_대수', 'LPG_하이브리드_예측']]], axis=1)
df5.head()

,연도,경유_대수,경유_예측,휘발유_대수,휘발유_예측,LPG_대수,LPG_예측
0,2019,9990643.00,NaN,11741283.00,NaN,1972573.00,NaN
1,2020,10023313.00,NaN,12346018.00,NaN,1954067.00,NaN
2,2021,9913104.00,NaN,12914379.00,NaN,1923815.00,NaN
3,2022,9980221.00,NaN,13552174.00,NaN,1936071.00,NaN
4,2023,NaN,9941451.50,NaN,14138722.00,NaN,1911692.00


In [ ]:
df5.columns

Index(['연도', '경유_대수', '경유_예측', '휘발유_대수', '휘발유_예측', 'LPG_대수', 'LPG_예측'], dtype='object')

In [ ]:
df5['테이블생성일자'] = today_date

In [ ]:
df5 = df5[[
    '테이블생성일자', 
    '연도', 
    '휘발유_하이브리드_대수', 
    '경유_하이브리드_대수', 
    'LPG_하이브리드_대수', 
    '휘발유_하이브리드_예측', 
    '경유_하이브리드_예측', 
    'LPG_하이브리드_예측'
    ]]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '휘발유_하이브리드_대수':'GSLH', 
    '경유_하이브리드_대수':'DSLH', 
    'LPG_하이브리드_대수':'LPGH', 
    '휘발유_하이브리드_예측':'GSLH_PRET', 
    '경유_하이브리드_예측':'DSLH_PRET', 
    'LPG_하이브리드_예측':'LPGH_PRET',
}

In [ ]:
STD_BD_HYBRD_CAR_PRET = df5.rename(columns=cdict)
STD_BD_HYBRD_CAR_PRET.columns

Index(['YR', 'GSL', 'DSL', 'LPG', 'GSL_PRET', 'DSL_PRET', 'LPG_PRET'], dtype='object')

In [ ]:
STD_BD_HYBRD_CAR_PRET.tail()

,YR,GSL,DSL,LPG,GSL_PRET,DSL_PRET,LPG_PRET
12,2031,NaN,NaN,NaN,18939549.20,9828271.50,1799885.60
13,2032,NaN,NaN,NaN,19539652.60,9814124.00,1785909.80
14,2033,NaN,NaN,NaN,20139756.00,9799976.50,1771934.00
15,2034,NaN,NaN,NaN,20739859.40,9785829.00,1757958.20
16,2035,NaN,NaN,NaN,21339962.80,9771681.50,1743982.40


### [출력] STD_BD_HYBRD_CAR_PRET

In [ ]:
expdf = STD_BD_HYBRD_CAR_PRET
table_nm = 'STD_BD_HYBRD_CAR_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## 내연차 연료, 등급, 연도별 차량 현황 예측
- 경유, 휘발유, LPG

In [143]:
# 2022년 차량 대수
grp1 = dfm2dgl.groupby(['fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', 'fuel', '배출가스등급', '차량대수']]

In [144]:
grp1.head()

,연도,fuel,배출가스등급,차량대수
0,2022,LPG,1,479004
1,2022,LPG,2,1176911
2,2022,LPG,3,238483
3,2022,LPG,4,24549
4,2022,LPG,5,436


In [145]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
grd_list = []
for fuel in grp1['fuel'].unique():
    for grd in grp1['배출가스등급'].unique():
        for yr in ['2019', '2020', '2021', '2022']:
            yr_list.append(yr)
            fuel_list.append(fuel)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list})

In [146]:
base.head()

,연도,fuel,배출가스등급
0,2019,LPG,1
1,2020,LPG,1
2,2021,LPG,1
3,2022,LPG,1
4,2019,LPG,2


In [147]:
# 연도별 등록대수
grp2 = dfm2dgl.groupby(['최초등록일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,배출가스등급,등록대수
0,1955,경유,5,1
1,1958,휘발유,5,1
2,1962,경유,5,8
3,1962,휘발유,5,1
4,1963,경유,5,2
...,...,...,...,...
445,2023,경유,3,42813
446,2023,경유,X,66194
447,2023,휘발유,1,72399
448,2023,휘발유,2,157626


In [148]:
# 연도별 말소대수
grp3 = errc2dgl.groupby(['변경일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,배출가스등급,말소대수
0,2019,LPG,1,5974
1,2019,LPG,2,27863
2,2019,LPG,3,42824
3,2019,LPG,4,3458
4,2019,LPG,5,3
...,...,...,...,...
59,2022,휘발유,2,116722
60,2022,휘발유,3,94401
61,2022,휘발유,4,11563
62,2022,휘발유,5,206


In [149]:
base1 = base.merge(grp1, on=['연도', 'fuel', '배출가스등급'], how='left')
base1

,연도,fuel,배출가스등급,차량대수
0,2019,LPG,1,NaN
1,2020,LPG,1,NaN
2,2021,LPG,1,NaN
3,2022,LPG,1,479004.00
4,2019,LPG,2,NaN
...,...,...,...,...
67,2022,휘발유,5,30012.00
68,2019,휘발유,X,NaN
69,2020,휘발유,X,NaN
70,2021,휘발유,X,NaN


In [150]:
base2 = base1.merge(grp2, on=['연도', 'fuel', '배출가스등급'], how='left')
base2

,연도,fuel,배출가스등급,차량대수,등록대수
0,2019,LPG,1,NaN,62978.00
1,2020,LPG,1,NaN,47598.00
2,2021,LPG,1,NaN,36005.00
3,2022,LPG,1,479004.00,16628.00
4,2019,LPG,2,NaN,65541.00
...,...,...,...,...,...
67,2022,휘발유,5,30012.00,NaN
68,2019,휘발유,X,NaN,2660.00
69,2020,휘발유,X,NaN,4611.00
70,2021,휘발유,X,NaN,5592.00


In [151]:
base3 = base2.merge(grp3, on=['연도', 'fuel', '배출가스등급'], how='left')
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,NaN,62978.00,5974.00
1,2020,LPG,1,NaN,47598.00,15887.00
2,2021,LPG,1,NaN,36005.00,23771.00
3,2022,LPG,1,479004.00,16628.00,10239.00
4,2019,LPG,2,NaN,65541.00,27863.00
...,...,...,...,...,...,...
67,2022,휘발유,5,30012.00,NaN,206.00
68,2019,휘발유,X,NaN,2660.00,303.00
69,2020,휘발유,X,NaN,4611.00,774.00
70,2021,휘발유,X,NaN,5592.00,1931.00


In [152]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    56
등록대수    25
말소대수     8
dtype: int64

In [153]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [154]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [155]:
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,428670.00,62978.00,5974.00
1,2020,LPG,1,460381.00,47598.00,15887.00
2,2021,LPG,1,472615.00,36005.00,23771.00
3,2022,LPG,1,479004.00,16628.00,10239.00
4,2019,LPG,2,1102289.00,65541.00,27863.00
...,...,...,...,...,...,...
67,2022,휘발유,5,30012.00,0.00,206.00
68,2019,휘발유,X,174659.00,2660.00,303.00
69,2020,휘발유,X,178496.00,4611.00,774.00
70,2021,휘발유,X,182157.00,5592.00,1931.00


In [156]:
die = base3.loc[base3['fuel'] == '경유', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['fuel'] == '휘발유', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['fuel'] == 'LPG', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)

In [157]:
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [158]:
die['배출가스등급'].unique()

array(['1', '2', '3', '4', '5', 'X'], dtype=object)

In [159]:
die1 = die.loc[die['배출가스등급'] == '1'].reset_index(drop=True)
die2 = die.loc[die['배출가스등급'] == '2'].reset_index(drop=True)
die3 = die.loc[die['배출가스등급'] == '3'].reset_index(drop=True)
die4 = die.loc[die['배출가스등급'] == '4'].reset_index(drop=True)
die5 = die.loc[die['배출가스등급'] == '5'].reset_index(drop=True)

In [160]:
gas1 = gas.loc[gas['배출가스등급'] == '1'].reset_index(drop=True)
gas2 = gas.loc[gas['배출가스등급'] == '2'].reset_index(drop=True)
gas3 = gas.loc[gas['배출가스등급'] == '3'].reset_index(drop=True)
gas4 = gas.loc[gas['배출가스등급'] == '4'].reset_index(drop=True)
gas5 = gas.loc[gas['배출가스등급'] == '5'].reset_index(drop=True)

In [161]:
lpg1 = lpg.loc[lpg['배출가스등급'] == '1'].reset_index(drop=True)
lpg2 = lpg.loc[lpg['배출가스등급'] == '2'].reset_index(drop=True)
lpg3 = lpg.loc[lpg['배출가스등급'] == '3'].reset_index(drop=True)
lpg4 = lpg.loc[lpg['배출가스등급'] == '4'].reset_index(drop=True)
lpg5 = lpg.loc[lpg['배출가스등급'] == '5'].reset_index(drop=True)

In [162]:
fit_d1 = np.polyfit(die1['연도'], die1['차량대수'], 1)
fit_d2 = np.polyfit(die2['연도'], die2['차량대수'], 1)
fit_d3 = np.polyfit(die3['연도'], die3['차량대수'], 1)
fit_d4 = np.polyfit(die4['연도'], die4['차량대수'], 1)
fit_d5 = np.polyfit(die5['연도'], die5['차량대수'], 1)

In [163]:
fit_g1 = np.polyfit(gas1['연도'], gas1['차량대수'], 1)
fit_g2 = np.polyfit(gas2['연도'], gas2['차량대수'], 1)
fit_g3 = np.polyfit(gas3['연도'], gas3['차량대수'], 1)
fit_g4 = np.polyfit(gas4['연도'], gas4['차량대수'], 1)
fit_g5 = np.polyfit(gas5['연도'], gas5['차량대수'], 1)

In [164]:
fit_l1 = np.polyfit(lpg1['연도'], lpg1['차량대수'], 1)
fit_l2 = np.polyfit(lpg2['연도'], lpg2['차량대수'], 1)
fit_l3 = np.polyfit(lpg3['연도'], lpg3['차량대수'], 1)
fit_l4 = np.polyfit(lpg4['연도'], lpg4['차량대수'], 1)
fit_l5 = np.polyfit(lpg5['연도'], lpg5['차량대수'], 1)

In [165]:
ad1, bd1 = fit_d1
ad2, bd2 = fit_d2
ad3, bd3 = fit_d3
ad4, bd4 = fit_d4
ad5, bd5 = fit_d5

In [166]:
ag1, bg1 = fit_g1
ag2, bg2 = fit_g2
ag3, bg3 = fit_g3
ag4, bg4 = fit_g4
ag5, bg5 = fit_g5

In [167]:
al1, bl1 = fit_l1
al2, bl2 = fit_l2
al3, bl3 = fit_l3
al4, bl4 = fit_l4
al5, bl5 = fit_l5

In [168]:
# 경유 1등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '1'
for yr in range(2023, 2036):
    pred = ad1 * yr + bd1
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 2등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '2'
for yr in range(2023, 2036):
    pred = ad2 * yr + bd2
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 3등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '3'
for yr in range(2023, 2036):
    pred = ad3 * yr + bd3
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 4등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '4'
for yr in range(2023, 2036):
    pred = ad4 * yr + bd4
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 5등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '5'
for yr in range(2023, 2036):
    pred = ad5 * yr + bd5
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})

In [169]:
# 휘발유 1등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '1'
for yr in range(2023, 2036):
    pred = ag1 * yr + bg1
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 2등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '2'
for yr in range(2023, 2036):
    pred = ag2 * yr + bg2
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 3등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '3'
for yr in range(2023, 2036):
    pred = ag3 * yr + bg3
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 4등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '4'
for yr in range(2023, 2036):
    pred = ag4 * yr + bg4
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 5등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '5'
for yr in range(2023, 2036):
    pred = ag5 * yr + bg5
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})

In [170]:
# LPG 1등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '1'
for yr in range(2023, 2036):
    pred = al1 * yr + bl1
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 2등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '2'
for yr in range(2023, 2036):
    pred = al2 * yr + bl2
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 3등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '3'
for yr in range(2023, 2036):
    pred = al3 * yr + bl3
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 4등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '4'
for yr in range(2023, 2036):
    pred = al4 * yr + bl4
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 5등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '5'
for yr in range(2023, 2036):
    pred = al5 * yr + bl5
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})

In [171]:
die1.head()

,연도,fuel,배출가스등급,차량대수
0,2019,경유,1,0.00
1,2020,경유,1,0.00
2,2021,경유,1,0.00
3,2022,경유,1,0.00


In [172]:
die_t = pd.concat([die1, die1_pred, die2, die2_pred, die3, die3_pred, die4, die4_pred, die5, die5_pred], ignore_index=True)
gas_t = pd.concat([gas1, gas1_pred, gas2, gas2_pred, gas3, gas3_pred, gas4, gas4_pred, gas5, gas5_pred], ignore_index=True)
lpg_t = pd.concat([lpg1, lpg1_pred, lpg2, lpg2_pred, lpg3, lpg3_pred, lpg4, lpg4_pred, lpg5, lpg5_pred], ignore_index=True)

In [173]:
die_t = die_t.rename(columns={'경유_예측':'차량예측'})
gas_t = gas_t.rename(columns={'휘발유_예측':'차량예측'})
lpg_t = lpg_t.rename(columns={'LPG_예측':'차량예측'})

In [174]:
df6 = pd.concat([die_t, gas_t, lpg_t], ignore_index=True)

In [175]:
df6.loc[df6['차량예측'] < 0, '차량예측'] = 0

In [176]:
df6.columns

Index(['연도', 'fuel', '배출가스등급', '차량대수', '차량예측'], dtype='object')

In [ ]:
df6['테이블생성일자'] = today_date

In [177]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    'fuel':'FUEL_CD', 
    '배출가스등급':'GRD', 
    '차량대수':'VHCL_MKCNT', 
    '차량예측':'VHCL_PRET', 
}

In [178]:
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET = df6.rename(columns=cdict)
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.columns

Index(['YR', 'FUEL_CD', 'GRD', 'VHCL_MKCNT', 'VHCL_PRET'], dtype='object')

In [179]:
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.head()

,YR,FUEL_CD,GRD,VHCL_MKCNT,VHCL_PRET
0,2019,경유,1,0.00,NaN
1,2020,경유,1,0.00,NaN
2,2021,경유,1,0.00,NaN
3,2022,경유,1,0.00,NaN
4,2023,경유,1,NaN,0.00


### [출력] STD_BD_FUEL_GRD_VHCL_CURSTT_PRET

In [ ]:
expdf = STD_BD_FUEL_GRD_VHCL_CURSTT_PRET
table_nm = 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## 무공해차 연료, 연도별 차량 현황 예측
- 전기, 수소

In [181]:
dfm2bh = dfm2.loc[(dfm2['fuel'] == '전기') | (dfm2['fuel'] == '수소')].reset_index(drop=True)
dfm2bh.shape

(437341, 25)

In [182]:
errc2bh = errc2.loc[(errc2['fuel'] == '전기') | (errc2['fuel'] == '수소')].reset_index(drop=True)
errc2bh.shape

(8112, 27)

In [183]:
# 2022년 차량 대수
grp1 = dfm2bh.groupby('fuel', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', 'fuel', '차량대수']]

In [184]:
grp1.head()

,연도,fuel,차량대수
0,2022,수소,29489
1,2022,전기,407852


In [185]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['fuel'].unique():
    for yr in ['2019', '2020', '2021', '2022']:
        yr_list.append(yr)
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list})

In [186]:
base.head()

,연도,fuel
0,2019,수소
1,2020,수소
2,2021,수소
3,2022,수소
4,2019,전기


In [187]:
# 연도별 등록대수
grp2 = dfm2bh.groupby(['최초등록일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,등록대수
0,2010,전기,10
1,2011,전기,106
2,2012,전기,369
3,2013,전기,489
4,2014,수소,5
5,2014,전기,1175
6,2015,수소,33
7,2015,전기,2767
8,2016,수소,45
9,2016,전기,3994


In [188]:
# 연도별 말소대수
grp3 = errc2bh.groupby(['변경일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,말소대수
0,2019,수소,5
1,2019,전기,593
2,2020,수소,19
3,2020,전기,1680
4,2021,수소,38
5,2021,전기,3950
6,2022,수소,37
7,2022,전기,1790


In [189]:
base1 = base.merge(grp1, on=['연도', 'fuel'], how='left')
base1

,연도,fuel,차량대수
0,2019,수소,NaN
1,2020,수소,NaN
2,2021,수소,NaN
3,2022,수소,29489.00
4,2019,전기,NaN
5,2020,전기,NaN
6,2021,전기,NaN
7,2022,전기,407852.00


In [190]:
base2 = base1.merge(grp2, on=['연도', 'fuel'], how='left')
base2

,연도,fuel,차량대수,등록대수
0,2019,수소,NaN,4163
1,2020,수소,NaN,5812
2,2021,수소,NaN,8516
3,2022,수소,29489.00,8136
4,2019,전기,NaN,33315
5,2020,전기,NaN,46263
6,2021,전기,NaN,100262
7,2022,전기,407852.00,128997


In [191]:
base3 = base2.merge(grp3, on=['연도', 'fuel'], how='left')
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,수소,NaN,4163,5
1,2020,수소,NaN,5812,19
2,2021,수소,NaN,8516,38
3,2022,수소,29489.00,8136,37
4,2019,전기,NaN,33315,593
5,2020,전기,NaN,46263,1680
6,2021,전기,NaN,100262,3950
7,2022,전기,407852.00,128997,1790


In [192]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    6
등록대수    0
말소대수    0
dtype: int64

In [193]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [194]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [195]:
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,수소,7119.00,4163,5
1,2020,수소,12912.00,5812,19
2,2021,수소,21390.00,8516,38
3,2022,수소,29489.00,8136,37
4,2019,전기,139750.00,33315,593
5,2020,전기,184333.00,46263,1680
6,2021,전기,280645.00,100262,3950
7,2022,전기,407852.00,128997,1790


In [196]:
bt = base3.loc[base3['fuel'] == '전기', ['연도', 'fuel', '차량대수']].reset_index(drop=True)
hy = base3.loc[base3['fuel'] == '수소', ['연도', 'fuel', '차량대수']].reset_index(drop=True)

In [197]:
bt['연도'] = bt['연도'].astype('int')
hy['연도'] = hy['연도'].astype('int')

In [198]:
fit1 = np.polyfit(bt['연도'], bt['차량대수'], 1)
fit2 = np.polyfit(hy['연도'], hy['차량대수'], 1)

In [199]:
a1, b1 = fit1
a2, b2 = fit2

In [200]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '전기'
for yr in range(2023, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
bt_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '전기_예측':pred_list})

In [201]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '수소'
for yr in range(2023, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
hy_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '수소_예측':pred_list})

In [202]:
bt_t = pd.concat([bt, bt_pred], ignore_index=True)
hy_t = pd.concat([hy, hy_pred], ignore_index=True)

In [203]:
bt_t = bt_t.rename(columns={'차량대수':'전기_대수'})
hy_t = hy_t.rename(columns={'차량대수':'수소_대수'})

In [204]:
hy_t.columns

Index(['연도', 'fuel', '수소_대수', '수소_예측'], dtype='object')

In [205]:
df7 = pd.concat([ bt_t[['연도', '전기_대수', '전기_예측']], hy_t[['수소_대수', '수소_예측']] ], axis=1)

In [206]:
df7

,연도,전기_대수,전기_예측,수소_대수,수소_예측
0,2019,139750.00,NaN,7119.00,NaN
1,2020,184333.00,NaN,12912.00,NaN
2,2021,280645.00,NaN,21390.00,NaN
3,2022,407852.00,NaN,29489.00,NaN
4,2023,NaN,478299.50,NaN,36624.50
5,2024,NaN,568361.30,NaN,44183.30
6,2025,NaN,658423.10,NaN,51742.10
7,2026,NaN,748484.90,NaN,59300.90
8,2027,NaN,838546.70,NaN,66859.70
9,2028,NaN,928608.50,NaN,74418.50


In [207]:
df7.columns

Index(['연도', '전기_대수', '전기_예측', '수소_대수', '수소_예측'], dtype='object')

In [ ]:
df7['테이블생성일자'] = today_date

In [208]:
df7 = df7[[
    '테이블생성일자', 
    '연도', 
    '전기_대수', 
    '수소_대수', 
    '전기_예측',
    '수소_예측', 
    ]]

In [209]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '전기_대수':'BTYCR', 
    '수소_대수':'HY', 
    '전기_예측':'BTYCR_PRET',
    '수소_예측':'HY_PRET',
}

In [210]:
STD_BD_ECO_CAR_PRET = df7.rename(columns=cdict)
STD_BD_ECO_CAR_PRET.columns

Index(['YR', 'BTYCR', 'HY', 'BTYCR_PRET', 'HY_PRET'], dtype='object')

### [출력] STD_BD_ECO_CAR_PRET

In [206]:
expdf = STD_BD_ECO_CAR_PRET
table_nm = 'STD_BD_ECO_CAR_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## 지역, 등급별 말소 차량 현황

In [54]:
errc2.shape

(3606827, 27)

In [55]:
errc2.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN_x', '제원관리번호', '차량연식',
       '차종', '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식',
       '엔진형식', '차량말소YN_y', '변경일자', '시도', '시군구', 'fuel', '지역', '변경일자_년',
       '변경일자_월', '변경일자_일', '시군구_수정'],
      dtype='object')

In [ ]:
errc2['테이블생성일자'] = today_date

In [208]:
df8 = errc2[[
    '테이블생성일자', 
    '법정동코드',
    '시도',
    '시군구', 
    '차대번호', 
    '변경일자', 
    '배출가스등급', 
    '연료'
    ]]
df8.columns

Index(['법정동코드', '차대번호', '변경일자', '배출가스등급', '연료'], dtype='object')

In [209]:
df8['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)', '전기', '휘발유 하이브리드', 'LPG 하이브리드', '수소',
       '경유 하이브리드'], dtype=object)

In [210]:
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '법정동코드':'STDG_CD',
    '시도':'CTPV',
    '시군구':'SGG', 
    '차대번호':'VIN', 
    '변경일자':'CHG_YMD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '연료':'FUEL_CD', 
}

In [211]:
STD_BD_GRD4_ERSR_RSLT = df8.rename(columns=chc_col)
STD_BD_GRD4_ERSR_RSLT.columns

Index(['STDG_CD', 'VIN', 'CHG_YMD', 'EXHST_GAS_GRD_CD', 'FUEL_CD'], dtype='object')

### [출력] STD_BD_GRD4_ERSR_RSLT

In [212]:
expdf = STD_BD_GRD4_ERSR_RSLT
table_nm = 'STD_BD_GRD4_ERSR_RSLT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

# code end